<a href="https://colab.research.google.com/github/VenkateshNni/ML/blob/main/Hybrid_prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Machine Learning Model for Stock Price Prediction


In this project, I developed a hybrid machine learning model that combines the strengths of LSTM (Long Short-Term Memory) networks and Linear Regression to predict stock prices more accurately. The model was trained on historical Apple stock market data, where LSTM captured complex sequential patterns, while Linear Regression modeled broader linear trends.

The Close price data was first scaled using MinMaxScaler, and sequences were created to feed into the LSTM model. A dual-path training approach was adopted:

The LSTM model was trained on past 60-day sequences to predict the next day's price.

The Linear Regression model used 3-day lagged features for trend estimation.

Finally, predictions from both models were combined using a weighted average (70% LSTM, 30% Linear Regression) to create a robust hybrid prediction. The model was also used to forecast future stock prices for the next 10 days, showcasing improved predictive performance compared to using a single model alone.

This hybrid approach illustrates how combining deep learning with traditional regression techniques can enhance forecasting accuracy in financial time series data.

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/content/apple_stock_data.csv')

In [ ]:
data.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [ ]:
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])  # Convert 'Date' column to datetime
data.set_index('Date', inplace=True)


In [ ]:
data=data[['Close']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data[['Close']])

<ipython-input-11-24eff1bed47e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Close'] = scaler.fit_transform(data[['Close']])


In [ ]:
import numpy as np
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(data['Close'].values, seq_length)

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.2454
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - loss: 0.0487
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0426
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.0162
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.0199
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0146
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0124
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0121
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0106
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0091
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0089
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0093
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0086
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0083
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0090
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━

In [ ]:
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

<ipython-input-17-0cdf358febc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Lag_1'] = data['Close'].shift(1)
<ipython-input-17-0cdf358febc8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Lag_2'] = data['Close'].shift(2)
<ipython-input-17-0cdf358febc8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [ ]:

 X_lin = data[['Lag_1', 'Lag_2', 'Lag_3']]
 y_lin = data['Close']

# Splitting data for Linear Regression
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

 X_train, X_test = X[:train_size], X[train_size:]


X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Adjust the number of predictions from the linear model to match the LSTM output
lin_predictions = lin_model.predict(X_test_lin[:X_test.shape[0]])  # Assuming lin_model exists
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

# Now you can combine predictions, ensuring they have the same length
hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)


lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)



hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [ ]:
lstm_future_predictions = []
last_sequence = X[-1].reshape(1, seq_length, 1)
for _ in range(10):
    lstm_pred = lstm_model.predict(last_sequence)[0, 0]
    lstm_future_predictions.append(lstm_pred)
    lstm_pred_reshaped = np.array([[lstm_pred]]).reshape(1, 1, 1)
    last_sequence = np.append(last_sequence[:, 1:, :], lstm_pred_reshaped, axis=1)
lstm_future_predictions = scaler.inverse_transform(np.array(lstm_future_predictions).reshape(-1, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [ ]:
recent_data = data['Close'].values[-3:]
lin_future_predictions = []
for _ in range(10):
    lin_pred = lin_model.predict(recent_data.reshape(1, -1))[0]
    lin_future_predictions.append(lin_pred)
    recent_data = np.append(recent_data[1:], lin_pred)
lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature n

In [ ]:
hybrid_future_predictions = (0.7 * lstm_future_predictions) + (0.3 * lin_future_predictions)

In [ ]:
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=10)
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM Predictions': lstm_future_predictions.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})
print(predictions_df)

                       Date  LSTM Predictions  Linear Regression Predictions  \
0 2024-11-02 00:00:00+00:00        231.647797                     230.355192   
1 2024-11-03 00:00:00+00:00        231.339905                     225.707291   
2 2024-11-04 00:00:00+00:00        231.092117                     222.703426   
3 2024-11-05 00:00:00+00:00        230.885574                     230.631535   
4 2024-11-06 00:00:00+00:00        230.707611                     225.486380   
5 2024-11-07 00:00:00+00:00        230.549484                     222.494588   
6 2024-11-08 00:00:00+00:00        230.405075                     230.930195   
7 2024-11-09 00:00:00+00:00        230.270264                     225.245599   
8 2024-11-10 00:00:00+00:00        230.142197                     222.284007   
9 2024-11-11 00:00:00+00:00        230.018936                     231.252375   

   Hybrid Model Predictions  
0                231.260015  
1                229.650115  
2                228.575508  